In [1]:
import requests
from bs4 import BeautifulSoup
import json
import sys
import time
from tqdm import tqdm

multiple advisors example

In [ ]:
mult_advisors = "https://www.genealogy.math.ndsu.nodak.edu/id.php?id=102043"

In [ ]:
first_pull_addr = "https://www.genealogy.math.ndsu.nodak.edu/id.php?id=230516"

In [ ]:
ex = "https://www.genealogy.math.ndsu.nodak.edu/id.php?id=42016"

In [ ]:
first_pull = requests.get(first_pull_addr)

In [ ]:
mult_adv = requests.get(mult_advisors)

In [ ]:
ex_pull = requests.get(ex)

In [ ]:
bs = BeautifulSoup(first_pull.text)

pull
- id
- name
- degree
- uni
- country
- year
- link to advisors
- links to advisees

In [ ]:
bs.find

In [ ]:
person_id = bs.find("a", text = "update form")["href"].split("=")[1].split("&")[0]
person_id

In [ ]:
name = bs.find("h2").text.strip()
name

In [ ]:
deg_uni_year = bs.find("span").text
deg_uni_year

In [ ]:
img_alt = "|".join([i["alt"] for i in bs.find_all("img")])
img_alt

In [ ]:
advs = [{"name": a.text, "link": a["href"]} for p in bs.find_all("p") for a in p.find_all("a") if a["href"][:2] == "id" and "Advisor" in p.text]
advs

In [ ]:
students = [{"name": a.text, "link": a["href"]} for t in bs.find_all("table") for a in t.find_all("a")]
students

In [ ]:
all_info = {
    "id": person_id,
    "name": name,
    "deg_uni_year": deg_uni_year,
    "img_alt": img_alt,
    "advisors": advs,
    "students": students
}

In [ ]:
all_info

In [ ]:
# json.dump([all_info], open("./math_gen.json", "w"))

In [ ]:
# mg = json.load(open("./math_gen.json", "r"))

In [ ]:
# mg

In [2]:
def get_info(link):
    addr = "https://www.genealogy.math.ndsu.nodak.edu/" + link
    pull = requests.get(addr)
    bs = BeautifulSoup(pull.text)
    
    person_id = bs.find("a", text = "update form")["href"].split("=")[1].split("&")[0]
    
    name = bs.find("h2").text.strip()
    
    deg_uni_year = bs.find("span").text
    
    img_alt = "|".join([i["alt"] for i in bs.find_all("img")])
    
    advs = [
        {
            "name": a.text,
            "link": a["href"]
        } for p in bs.find_all("p") for a in p.find_all("a")\
        if a["href"][:2] == "id" and ("Advisor" in p.text or "Promotor" in p.text)]
    
    students = [{"name": a.text, "link": a["href"]} for t in bs.find_all("table") for a in t.find_all("a")]
    
    return {
        "id": person_id,
        "name": name,
        "deg_uni_year": deg_uni_year,
        "img_alt": img_alt,
        "advisors": advs,
        "students": students}

In [3]:
def find_next(mg, ind, checked_ind = [], nr = 1):
    elt = mg[ind]
    
    # if nr > 4:
    #     return None
    
    # search through students
    for student in elt["students"]:
        # try to find the student in the list
        search_mg = [i for i, e in enumerate(mg) if e["id"] == student["link"].split("=")[1]]
        if len(search_mg) == 0:
            return student["link"]
        else:
            if not search_mg[0] in checked_ind:
                student_next = find_next(mg, search_mg[0], checked_ind + [ind], nr + 1)
                if student_next:
                    return student_next
    
    # search through advisors
    for advisor in elt["advisors"]:
        # try to find the advisor in the list
        search_mg = [i for i, e in enumerate(mg) if e["id"] == advisor["link"].split("=")[1]]
        if len(search_mg) == 0:
            return advisor["link"]
        else:
            if not search_mg[0] in checked_ind:
                advisor_next = find_next(mg, search_mg[0], checked_ind + [ind], nr + 1)
                if advisor_next:
                    return advisor_next

    return None

In [4]:
# find_next(mg, 0)

Define the full process

In [5]:
def add_to_math_gen(path):
    # get current math gen data
    mg = json.load(open(path, "r"))
    
    # find the next link to add
    # search through a few previous elements to find a non-null next
    next_link = None
    search_ind = 1
    while not next_link and search_ind < 5:
        next_link = find_next(mg, len(mg) - search_ind)
        search_ind += 1
        
    if next_link:
    
        # print("grabbing from " + next_link)

        # get info from that link
        next_link_info = get_info(next_link)

        # print("got " + next_link_info["name"])

        # add info to math gen data
        next_mg = mg + [next_link_info]

        # print("data size " + str(len(next_mg)))

        # write to file
        json.dump(next_mg, open(path, "w"))

In [ ]:
for i in tqdm(range(10000)):
    add_to_math_gen("./math_gen2.json")
    time.sleep(1)

  0%|          | 18/10000 [00:44<6:49:10,  2.46s/it]